### Generate Embeddings

In [ ]:
import ollama
import chromadb
from pypdf import PdfReader

documents = [
  
]
reader = PdfReader("file2.pdf")
number_of_pages = len(reader.pages)
for i in range(len(reader.pages)):
    page = reader.pages[i]
    documents.append(page.extract_text())

client = chromadb.Client()
try:
    collection = client.create_collection(name="docs")
except:
    pass

# store each document in a vector embedding database
for i, d in enumerate(documents):
  response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
  embedding = response["embedding"]
  collection.add(
    ids=[str(i)],
    embeddings=[embedding],
    documents=[d]
  )



## Interface

In [ ]:
import gradio as gr
import ollama
import requests
from serpapi import GoogleSearch

# Define your SerpAPI key
SERPAPI_API_KEY = ""

# Function to search Google using SerpAPI
def google_search(query):
    search = GoogleSearch({
        "q": query,
        "api_key": SERPAPI_API_KEY,
        "google_domain": "google.co.in"
    })
    results = search.get_dict()
    return results['organic_results'] if 'organic_results' in results else []


def chatbot(prompt, chat_history):
    # Generate an embedding for the prompt
    response = ollama.embeddings(
        prompt=prompt,
        model="mxbai-embed-large"
    )
    
    results = collection.query(
        query_embeddings=[response["embedding"]],
        n_results=3
    )
    
    # Extract the data from the result
    data = results['documents'][0][0]
    
    # Fetch real-time data from Google Search
    google_results = google_search(prompt)
    
    # Combine data from knowledge base and Google search results
    combined_data = f"Knowledge base data: '{data}'. Google search results: {google_results[:3]}. "
    
    # Generate a response using the "mistral-small" model with streaming
    full_response = ""
    for chunk in ollama.generate(
#         model="llama3.1",
        model="mistral-small",
        prompt=f"Using this data: '{combined_data}' and your knowledge, respond to this prompt with explanation of why this answer only and answer should be given first then explaination: {prompt}",
#         prompt=f"Using this data: '{combined_data}' and your knowledge, respond to this prompt:'{prompt}' and also highlight the answer in bold so it is visible",
        stream=True
    ):
        full_response += chunk['response']
        yield chat_history + [(prompt, full_response)]
    
    # Append user input and model output to the chat history
    chat_history.append((prompt, full_response))
    yield chat_history

# Create the Gradio chatbot interface
with gr.Blocks(title="RAG with Google Search") as chatbot_ui:
    gr.Markdown("""<h1><center>Urban Guide: A RAG-Based City Assistant Using LLMs and Agents</center></h1>""")
    chatbot_history = gr.Chatbot()  # For displaying the chat-like conversation
    user_input = gr.Textbox(show_label=False, placeholder="Type your message here...")  # Textbox for user input
    submit_button = gr.Button("Send")  # Submit button
    
    # Define the interaction flow
    submit_button.click(chatbot, inputs=[user_input, chatbot_history], outputs=chatbot_history)
    user_input.submit(chatbot, inputs=[user_input, chatbot_history], outputs=chatbot_history)

# Launch the interface
chatbot_ui.launch(share=True)

## Without RAG

In [ ]:
import gradio as gr
import ollama
# import requests
# from serpapi import GoogleSearch

# Define your SerpAPI key
# SERPAPI_API_KEY = ""

# # Function to search Google using SerpAPI
# def google_search(query):
#     search = GoogleSearch({
#         "q": query,
#         "api_key": SERPAPI_API_KEY,
#         "google_domain": "google.co.in"
#     })
#     results = search.get_dict()
#     return results['organic_results'] if 'organic_results' in results else []


def chatbot(prompt, chat_history):
    # Generate an embedding for the prompt
#     response = ollama.embeddings(
#         prompt=prompt,
#         model="mxbai-embed-large"
#     )
    
#     results = collection.query(
#         query_embeddings=[response["embedding"]],
#         n_results=1
#     )
    
#     # Extract the data from the result
#     data = results['documents'][0][0]
    
    # Fetch real-time data from Google Search
#     google_results = google_search(prompt)
    
    # Combine data from knowledge base and Google search results
#     combined_data = f"Knowledge base data: '{data}'. Google search results: {google_results[:3]}."
    
    # Generate a response using the "mistral-small" model with streaming
    full_response = ""
    for chunk in ollama.generate(
#         model="llama3.1",
        model="qwen2.5-coder:32b",
        prompt=prompt,
#         prompt=f"Using this data: '{combined_data}' and your knowledge, respond to this prompt:'{prompt}' and also highlight the answer in bold so it is visible",
        stream=True
    ):
        full_response += chunk['response']
        yield chat_history + [(prompt, full_response)]
    
    # Append user input and model output to the chat history
    chat_history.append((prompt, full_response))
    yield chat_history

# Create the Gradio chatbot interface
with gr.Blocks(title="Qwen Coder") as chatbot_ui:
    gr.Markdown("""<h1><center>Coder</center></h1>""")
    chatbot_history = gr.Chatbot()  # For displaying the chat-like conversation
    user_input = gr.Textbox(show_label=False, placeholder="Type your message here...")  # Textbox for user input
    submit_button = gr.Button("Send")  # Submit button
    
    # Define the interaction flow
    submit_button.click(chatbot, inputs=[user_input, chatbot_history], outputs=chatbot_history)
    user_input.submit(chatbot, inputs=[user_input, chatbot_history], outputs=chatbot_history)

# Launch the interface
chatbot_ui.launch(share=True)

### Gradio Interface

In [ ]:
import gradio as gr
import ollama

# Define the function that will be used in the Gradio interface
def chatbot(prompt, chat_history):
    # Generate an embedding for the prompt
    response = ollama.embeddings(
        prompt=prompt,
        model="mxbai-embed-large"
    )
    
    # Query the collection to get the most relevant document
    results = collection.query(
        query_embeddings=[response["embedding"]],
        n_results=1
    )
    
    # Extract the data from the result
    data = results['documents'][0][0]
    
    # Generate a response using the "mistral-small" model
    output = ollama.generate(
        model="mistral-small",
        prompt=f"Using this data: '{data}' and your knowledge. Respond to this prompt: {prompt}"
    )
    
    # Append user input and model output to the chat history
    chat_history.append((prompt, output['response']))
    return chat_history, chat_history

# Create the Gradio chatbot interface
with gr.Blocks(title="Urban Guide") as chatbot_ui:
    gr.Markdown("""<h1><center> Urban Guide (Custom Knowledge base QA with LLMs) </center></h1>""")
    chatbot_history = gr.Chatbot()  # For displaying the chat-like conversation
    user_input = gr.Textbox(show_label=False, placeholder="Type your message here...")  # Textbox for user input
    submit_button = gr.Button("Send")  # Submit button
    
    gr.Markdown('<center><img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*diTLYX2NBstoDrSjLglM5g.png"/></center>')

    # Define the interaction flow
    submit_button.click(chatbot, inputs=[user_input, chatbot_history], outputs=[chatbot_history, chatbot_history])
    user_input.submit(chatbot, inputs=[user_input, chatbot_history], outputs=[chatbot_history, chatbot_history])

# Launch the interface
chatbot_ui.launch(share=True)

### Testing

In [ ]:
import gradio as gr

def greet(name):
    return f"Hello {name}!"

iface = gr.Interface(fn=greet, inputs="text", outputs="text")
iface.launch()


In [ ]:
import ollama
response = ollama.chat(model='mistral-small', messages=[
  {
    'role': 'user',
    'content': 'give me a simple code for retrival augmented generation using langchain and ollama',
  },
])
print(response['message']['content'])

In [ ]:
import gradio as gr
import ollama
import requests
import wikipedia
from serpapi import GoogleSearch  # Import SerpAPI for Google search

# Define your SerpAPI key
SERPAPI_API_KEY = ""

# Function to search Google using SerpAPI
def google_search(query):
    search = GoogleSearch({
        "q": query,
        "api_key": SERPAPI_API_KEY,
        "google_domain": "google.co.in"
    })
    results = search.get_dict()
    return results['organic_results'] if 'organic_results' in results else []

# Define the function that will be used in the Gradio interface
def chatbot(prompt, chat_history):
    # Generate an embedding for the prompt
    response = ollama.embeddings(
        prompt=prompt,
        model="mxbai-embed-large"
    )
    
    # Query the collection to get the most relevant document
    results = collection.query(
        query_embeddings=[response["embedding"]],
        n_results=1
    )
    
    # Extract the data from the result
    data = results['documents'][0][0]
    
    if "who is" in prompt:
        wikidata = wikipedia.summary("Ratan Tata", 3)
    else:
        wikidata = ""
    
    # Fetch real-time data from Google Search
    google_results = google_search(prompt)
    
    # Combine data from knowledge base and Google search results
    combined_data = f"Knowledge base data: '{data}'. Google search results: {google_results[:3]}. Wikipedia Data: {wikidata}"
    
    # Generate a response using the "mistral-small" model
    output = ollama.generate(
        model="mistral-small",
        prompt=f"Using this data: '{combined_data}' and your knowledge, respond to this prompt with explaination of why this answer only: {prompt}"
    )
    
    # Append user input and model output to the chat history
    chat_history.append((prompt, output['response']))
    return chat_history, chat_history

# Create the Gradio chatbot interface
with gr.Blocks(title="RAG with Google Search") as chatbot_ui:
    gr.Markdown("""<h1><center>Custom Knowledge Base QA with LLMs and Internet Data</center></h1>""")
    chatbot_history = gr.Chatbot()  # For displaying the chat-like conversation
    user_input = gr.Textbox(show_label=False, placeholder="Type your message here...")  # Textbox for user input
    submit_button = gr.Button("Send")  # Submit button
    
#     gr.Markdown('<center><img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*diTLYX2NBstoDrSjLglM5g.png"/></center>')

    # Define the interaction flow
    submit_button.click(chatbot, inputs=[user_input, chatbot_history], outputs=[chatbot_history, chatbot_history])
    user_input.submit(chatbot, inputs=[user_input, chatbot_history], outputs=[chatbot_history, chatbot_history])

# Launch the interface
chatbot_ui.launch(share=True)


In [ ]:
import gradio as gr
import ollama
import requests
from serpapi import GoogleSearch

# Define your SerpAPI key
SERPAPI_API_KEY = "your_serpapi_api_key"

# Function to search Google using SerpAPI
def google_search(query):
    search = GoogleSearch({
        "q": query,
        "api_key": SERPAPI_API_KEY
    })
    results = search.get_dict()
    return results['organic_results'] if 'organic_results' in results else []

# Function to search Google Maps using SerpAPI
def google_maps_search(query, location=""):
    search_params = {
        "q": query,
        "location": location,  # Optional location parameter
        "api_key": SERPAPI_API_KEY,
        "engine": "google_maps"  # Use Google Maps search engine
    }
    search = GoogleSearch(search_params)
    results = search.get_dict()
    return results['local_results'] if 'local_results' in results else []

# Define the function that will be used in the Gradio interface
def chatbot(prompt, chat_history):
    # Generate an embedding for the prompt
    response = ollama.embeddings(
        prompt=prompt,
        model="mxbai-embed-large"
    )
    
    # Query the collection to get the most relevant document
    results = collection.query(
        query_embeddings=[response["embedding"]],
        n_results=1
    )
    
    # Check if a document is available in the knowledge base
    if results['documents']:
        # Extract the data from the knowledge base
        data = results['documents'][0][0]
        source = "Knowledge base"
    else:
        # If no document is found, try Google Maps for location-based queries
        google_maps_results = google_maps_search(prompt)
        if google_maps_results:
            # Extract relevant location from Google Maps results
            data = f"Here is a relevant location: {google_maps_results[0]['title']} at {google_maps_results[0]['address']}"
            source = "Google Maps"
        else:
            # If no location is found, perform a regular Google search
            google_results = google_search(prompt)
            if google_results:
                data = google_results[0]['snippet']  # Extract snippet from Google search
                source = "Google search"
            else:
                return chat_history, chat_history.append((prompt, "Sorry, no data available from both the knowledge base and Google search."))
    
    # Combine data and source
    combined_data = f"Using {source} data: '{data}'."

    # Generate a response using the "mistral-small" model
    output = ollama.generate(
        model="mistral-small",
        prompt=f"{combined_data} Respond to this prompt: {prompt}"
    )
    
    # Append user input and model output to the chat history
    chat_history.append((prompt, output['response']))
    return chat_history, chat_history

# Create the Gradio chatbot interface
with gr.Blocks(title="RAG with Google Search and Maps") as chatbot_ui:
    gr.Markdown("""<h1><center>Custom Knowledge Base QA with LLMs, Google Search, and Maps</center></h1>""")
    chatbot_history = gr.Chatbot()  # For displaying the chat-like conversation
    user_input = gr.Textbox(show_label=False, placeholder="Type your message here...")  # Textbox for user input
    submit_button = gr.Button("Send")  # Submit button
    
#     gr.Markdown('<center><img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*diTLYX2NBstoDrSjLglM5g.png"/></center>')

    # Define the interaction flow
    submit_button.click(chatbot, inputs=[user_input, chatbot_history], outputs=[chatbot_history, chatbot_history])
    user_input.submit(chatbot, inputs=[user_input, chatbot_history], outputs=[chatbot_history, chatbot_history])

# Launch the interface
chatbot_ui.launch(share=True)


In [ ]:
import pywhatkit as kit

# Fetch a summary from Wikipedia
kit.info("Python programming language", web=False)


In [ ]:
import gradio as gr
import ollama
import requests
from serpapi import GoogleSearch  # Import SerpAPI for Google search

# Define your SerpAPI key
SERPAPI_API_KEY = "apikey"

# Function to search Google using SerpAPI
def google_search(query):
    search = GoogleSearch({
        "q": query,
        "api_key": SERPAPI_API_KEY,
        "google_domain": "google.co.in"
    })
    results = search.get_dict()
    return results['organic_results'] if 'organic_results' in results else []

# Define the function that will be used in the Gradio interface
def chatbot(prompt, chat_history):
    # Generate an embedding for the prompt
    response = ollama.embeddings(
        prompt=prompt,
        model="mxbai-embed-large"
    )
    
    # Query the collection to get the most relevant document
    results = collection.query(
        query_embeddings=[response["embedding"]],
        n_results=1
    )
    
    # Extract the data from the result
    data = results['documents'][0][0]
    
    
    # Fetch real-time data from Google Search
    google_results = google_search(prompt)
    
    # Combine data from knowledge base and Google search results
    combined_data = f"Knowledge base data: '{data}'. Google search results: {google_results[:3]}."
    
    # Generate a response using the "mistral-small" model
    output = ollama.generate(
        model="mistral-small",
        prompt=f"Using this data: '{combined_data}' and your knowledge, respond to this prompt with explaination of why this answer only: {prompt}"
    )
    
    # Append user input and model output to the chat history
    chat_history.append((prompt, output['response']))
    return chat_history, chat_history

# Create the Gradio chatbot interface
with gr.Blocks(title="RAG with Google Search") as chatbot_ui:
    gr.Markdown("""<h1><center>Urban Guide: A RAG-Based City Assistant Using LLMs and Agents</center></h1>""")
    chatbot_history = gr.Chatbot()  # For displaying the chat-like conversation
    user_input = gr.Textbox(show_label=False, placeholder="Type your message here...")  # Textbox for user input
    submit_button = gr.Button("Send")  # Submit button
    
#     gr.Markdown('<center><img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*diTLYX2NBstoDrSjLglM5g.png"/></center>')

    # Define the interaction flow
    submit_button.click(chatbot, inputs=[user_input, chatbot_history], outputs=[chatbot_history, chatbot_history])
    user_input.submit(chatbot, inputs=[user_input, chatbot_history], outputs=[chatbot_history, chatbot_history])

# Launch the interface
chatbot_ui.launch(share=True)


# Currently Working

In [ ]:
import gradio as gr
import ollama
import requests
from serpapi import GoogleSearch

# Define your SerpAPI key
SERPAPI_API_KEY = ""

# Function to search Google using SerpAPI
def google_search(query):
    search = GoogleSearch({
        "q": query,
        "api_key": SERPAPI_API_KEY,
        "google_domain": "google.co.in"
    })
    results = search.get_dict()
    return results['organic_results'] if 'organic_results' in results else []


def chatbot(prompt, chat_history):
    # Generate an embedding for the prompt
    response = ollama.embeddings(
        prompt=prompt,
        model="mxbai-embed-large"
    )
    
    results = collection.query(
        query_embeddings=[response["embedding"]],
        n_results=1
    )
    
    # Extract the data from the result
    data = results['documents'][0][0]
    
    # Fetch real-time data from Google Search
    google_results = google_search(prompt)
    
    # Combine data from knowledge base and Google search results
    combined_data = f"Knowledge base data: '{data}'. Google search results: {google_results[:3]}."
    
    # Generate a response using the "mistral-small" model with streaming
    full_response = ""
    for chunk in ollama.generate(
#         model="llama3.1",
        model="mistral-small",
        prompt=f"Using this data: '{combined_data}' and your knowledge, respond to this prompt with explanation of why this answer only: {prompt}",
        stream=True
    ):
        full_response += chunk['response']
        yield chat_history + [(prompt, full_response)]
    
    # Append user input and model output to the chat history
    chat_history.append((prompt, full_response))
    yield chat_history

# Create the Gradio chatbot interface
with gr.Blocks(title="RAG with Google Search") as chatbot_ui:
    gr.Markdown("""<h1><center>Urban Guide: A RAG-Based City Assistant Using LLMs and Agents</center></h1>""")
    chatbot_history = gr.Chatbot()  # For displaying the chat-like conversation
    user_input = gr.Textbox(show_label=False, placeholder="Type your message here...")  # Textbox for user input
    submit_button = gr.Button("Send")  # Submit button
    
    # Define the interaction flow
    submit_button.click(chatbot, inputs=[user_input, chatbot_history], outputs=chatbot_history)
    user_input.submit(chatbot, inputs=[user_input, chatbot_history], outputs=chatbot_history)

# Launch the interface
chatbot_ui.launch(share=True)

# With Context 

In [ ]:
import gradio as gr
import ollama
import requests
from serpapi import GoogleSearch

# Define your SerpAPI key
SERPAPI_API_KEY = ""

# Function to search Google using SerpAPI
def google_search(query):
    search = GoogleSearch({
        "q": query,
        "api_key": SERPAPI_API_KEY,
        "google_domain": "google.co.in"
    })
    results = search.get_dict()
    return results['organic_results'] if 'organic_results' in results else []

class ContextualChatbot:
        
    def generate_response(self, prompt: str, chat_history: List[Tuple[str, str]]):
        # Build the conversation context into the prompt instead
        context_prompt = ""
        for user_msg, assistant_msg in chat_history:
            context_prompt += f"User: {user_msg}\nAssistant: {assistant_msg}\n\n"
        
        # Get data from Knowledgebase
        response = ollama.embeddings(
            prompt=prompt,
            model="mxbai-embed-large"
        )

        results = collection.query(
            query_embeddings=[response["embedding"]],
            n_results=1
        )

        # Extract the data from the result
        data = results['documents'][0][0]
        google_results = google_search(prompt)
        combined_data = f"Knowledge base data: '{data}'. Data from internet: {google_results[:3]}."
        # Add current prompt with context
        prompt=f"Using this data: '{combined_data}' and your knowledge, respond to this prompt with explanation of why this answer only: {prompt}",
        
        # Generate response
        full_response = ""
        for chunk in ollama.generate(
            model="mistral-small",
            prompt=full_prompt,
            system=system_prompt,
            stream=True,
            options={
                "temperature": 0.7,
                "top_p": 0.9,
            }
        ):
            full_response += chunk['response']
            yield chat_history + [(prompt, full_response)]
        
        # Update chat history
        chat_history.append((prompt, full_response))
        yield chat_history

# Create chatbot instance
chatbot = ContextualChatbot()

# Create the Gradio interface
with gr.Blocks(title="RAG with Google Search") as chatbot_ui:
    gr.Markdown("""<h1><center>Urban Guide: A RAG-Based City Assistant Using LLMs and Agents</center></h1>""")
    chatbot_history = gr.Chatbot()
    user_input = gr.Textbox(
        show_label=False, 
        placeholder="Give Your queries here...",
        scale=4
    )
    submit_button = gr.Button("Send", scale=1)
#     clear_button = gr.Button("Clear Chat")
    
    
    # Define the interaction flow
    submit_button.click(
        chatbot.generate_response, 
        inputs=[user_input, chatbot_history], 
        outputs=chatbot_history
    )
    user_input.submit(
        chatbot.generate_response, 
        inputs=[user_input, chatbot_history], 
        outputs=chatbot_history
    )
#     clear_button.click(lambda: None, None, chatbot_history, queue=False)

# Launch the interface
chatbot_ui.launch(share=True)

In [ ]:
import gradio as gr
import ollama
# import requests


def chatbot(prompt, chat_history):
    
    # Generate a response using the "mistral-small" model with streaming
    full_response = ""
    for chunk in ollama.generate(
#         model="llama3.1",
        model="mistral-small",
#         context=
        prompt=prompt,
        stream=True
    ):
        full_response += chunk['response']
        yield chat_history + [(prompt, full_response)]
    
    # Append user input and model output to the chat history
    chat_history.append((prompt, full_response))
    yield chat_history

# Create the Gradio chatbot interface
with gr.Blocks(title="Code") as chatbot_ui:

    chatbot_history = gr.Chatbot()  # For displaying the chat-like conversation
    user_input = gr.Textbox(show_label=False, placeholder="Type your message here...")  # Textbox for user input
    submit_button = gr.Button("Send")  # Submit button
    
    # Define the interaction flow
    submit_button.click(chatbot, inputs=[user_input, chatbot_history], outputs=chatbot_history)
    user_input.submit(chatbot, inputs=[user_input, chatbot_history], outputs=chatbot_history)

# Launch the interface
chatbot_ui.launch(share=True)

In [ ]:
import gradio as gr
import ollama
import json

# Function to load chat history
def load_history():
    try:
        with open('chat_history.json', 'r') as f:
            return json.load(f)
    except:
        return []

# Function to save chat history
def save_history(history):
    with open('chat_history.json', 'w') as f:
        json.dump(history, f)

def chatbot(prompt, chat_history):
    if chat_history is None:
        chat_history = []
    
    # Generate a response using the "mistral-small" model with streaming
    full_response = ""
    for chunk in ollama.generate(
        model="mistral-small",
        prompt=prompt,
        stream=True
    ):
        full_response += chunk['response']
        new_history = chat_history + [(prompt, full_response)]
        save_history(new_history)  # Save after each update
        yield new_history
    
    # Final update
    chat_history.append((prompt, full_response))
    save_history(chat_history)  # Save final state
    return chat_history

def clear_history():
    save_history([])  # Clear saved history
    return []

# Create the Gradio chatbot interface
with gr.Blocks(title="Code") as chatbot_ui:
    # Initialize with saved history
    chatbot_history = gr.Chatbot(
        value=load_history(),
        elem_id="chatbot",
        height=400
    )
    
    with gr.Row():
        user_input = gr.Textbox(
            show_label=False, 
            placeholder="Type your message here...",
            scale=4,
            container=False
        )
        submit_button = gr.Button("Send", scale=1)
    
    clear_button = gr.Button("Clear History")

    # Define the interaction flow
    submit_button.click(
        chatbot,
        inputs=[user_input, chatbot_history],
        outputs=chatbot_history
    )

    user_input.submit(
        chatbot,
        inputs=[user_input, chatbot_history],
        outputs=chatbot_history
    )
    
    clear_button.click(
        clear_history,
        outputs=chatbot_history
    )

# Launch the interface
chatbot_ui.launch(share=True)

## Without Streaming response without rag


In [ ]:
import gradio as gr
import ollama

def chatbot(prompt, chat_history):
    full_response = ""
    response = ollama.generate(
        model="deepseek-r1:32b",
        prompt=prompt
    )
    full_response += response['response']

    # Append user input and model output to the chat history
    chat_history.append((prompt, full_response))
    return chat_history

# Create the Gradio chatbot interface
with gr.Blocks(title="Qwen Coder") as chatbot_ui:
    gr.Markdown("""<h1><center>Coder</center></h1>""")
    chatbot_history = gr.Chatbot()  # For displaying the chat-like conversation
    markdown_output = gr.Markdown()  # For displaying Markdown content
    user_input = gr.Textbox(show_label=False, placeholder="Type your message here...")  # Textbox for user input
    submit_button = gr.Button("Send")  # Submit button

    # Define the interaction flow
    def update_ui(prompt, chat_history):
        result = chatbot(prompt, chat_history)
        markdown_content = result[-1][1]  # Extract the latest response as Markdown
        return result, markdown_content

    submit_button.click(update_ui, inputs=[user_input, chatbot_history], outputs=[chatbot_history, markdown_output])
    user_input.submit(update_ui, inputs=[user_input, chatbot_history], outputs=[chatbot_history, markdown_output])

# Launch the interface
chatbot_ui.launch(share=True)


# With Context